In [1]:
import wandb
import pandas as pd
from tqdm.auto import tqdm

/Users/razbesaleli/fewshot-sql-descriptions/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_json('data.json')
df.head()

,question,query,generation,retrieval_method,nshot,masked,no_literals,precision,recall,f1
0,what is warfarin sodium 2.5 mg po tabs's way o...,select distinct medication.routeadmin from med...,what is the ingesting method of warfarin sodium?,column_jaccard,1,True,False,0.808309,0.662931,0.728437
1,"what is the method for ingestion of ns 1,000 ml?",select distinct medication.routeadmin from med...,what is the ingesting method of niacin?,column_jaccard,1,True,False,0.728178,0.666734,0.696103
2,how is atorvastatin calcium 80 mg po tabs taken?,select distinct medication.routeadmin from med...,what is the name of the medication containing ...,column_jaccard,1,True,False,0.716453,0.668118,0.691442
3,what is metoprolol succinate er 50 mg po tb24'...,select distinct medication.routeadmin from med...,what is the prescribing doctor's name and the ...,column_jaccard,1,True,False,0.552028,0.427494,0.481845
4,tell me the price of a procedure called agent ...,select distinct cost.cost from cost where cost...,what is the price of a procedure named agent s...,column_jaccard,1,True,False,0.872670,0.864234,0.868431


In [3]:
df = (
    df[df['no_literals'] == False]
    .drop(labels=['no_literals'], axis=1)
    .reset_index(drop=True)
    )

In [4]:
# add zeroshot to all retrieval methods
zeroshot = pd.DataFrame(df[df['nshot'] == 0])
for retrieval_method in df['retrieval_method'].unique():
    if retrieval_method != 'random':
        zeroshot['retrieval_method'] = [retrieval_method] * len(zeroshot)
        df = pd.concat([df, zeroshot])
        
print(len(df))
print(df['retrieval_method'].value_counts())
assert len(set(df['retrieval_method'].value_counts().to_list())) == 1
df = df.sort_values(by=['retrieval_method', 'nshot']).reset_index(drop=True)
df['generation'] = df['generation'].map(lambda i: i.strip())
df.head()

18120
column_jaccard    6040
tfidf             6040
random            6040
Name: retrieval_method, dtype: int64


,question,query,generation,retrieval_method,nshot,masked,precision,recall,f1
0,what is warfarin sodium 2.5 mg po tabs's way o...,select distinct medication.routeadmin from med...,The query extracts all of the patient's medica...,column_jaccard,0,False,0.590455,0.661140,0.623802
1,"what is the method for ingestion of ns 1,000 ml?",select distinct medication.routeadmin from med...,This query selects all the rows from the medic...,column_jaccard,0,False,0.563709,0.675585,0.614598
2,how is atorvastatin calcium 80 mg po tabs taken?,select distinct medication.routeadmin from med...,This query is looking for all medications with...,column_jaccard,0,False,0.653431,0.766081,0.705286
3,what is metoprolol succinate er 50 mg po tb24'...,select distinct medication.routeadmin from med...,This query presents a list of distinct medicat...,column_jaccard,0,False,0.696146,0.752771,0.723352
4,tell me the price of a procedure called agent ...,select distinct cost.cost from cost where cost...,This query will return all of the different co...,column_jaccard,0,False,0.677709,0.668955,0.673304


In [5]:
next(df.iterrows())[1].to_dict()

{'question': "what is warfarin sodium 2.5 mg po tabs's way of ingesting it?",
 'query': "select distinct medication.routeadmin from medication where medication.drugname = 'warfarin sodium 2.5 mg po tabs'",
 'generation': "The query extracts all of the patient's medication records that have the warfarin sodium 2.5 mg po tabs drug name in them.",
 'retrieval_method': 'column_jaccard',
 'nshot': 0,
 'masked': False,
 'precision': 0.5904552937,
 'recall': 0.6611401439,
 'f1': 0.6238017082}

In [6]:
wandb.login(key=open('.wandb_api_key').read())

PROJECT_NAME = 'fewshot-sql-descriptions'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: besalelir1 (besalelir1-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/razbesaleli/.netrc


In [7]:
for retrieval_method, by_retrieval_method in df.groupby('retrieval_method'):
    for is_masked, by_mask in by_retrieval_method.groupby('masked'):
        for nshot, dff in by_mask.groupby('nshot'):
            run_name = f"{retrieval_method}-{'' if is_masked else 'un'}masked-{nshot}shot"
            
            config = {
                'is_masked': is_masked,
                'retrieval_method': retrieval_method,
                'nshot': nshot
            }

            with wandb.init(project=PROJECT_NAME, name=run_name, config=config) as run:
                for _, row in tqdm(dff.iterrows(), total=len(dff)):
                    run.log(row.to_dict())

100%|██████████| 755/755 [00:00<00:00, 10294.79it/s]


f1,▅▂▄▆▄▄▃▁▁▂▄▃▆▂█▁▂▄▁▅▂▄▃▃▆▃▂▅▅▅▄▆▅▄▃▄▅▂▅▃
masked,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nshot,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▄▃▄▆▃▄▃▁▁▂▅▃▅▃█▁▃▃▂▆▃▄▃▂█▄▂▅▄▅▃▆▄▄▃▃▅▂▅▂
recall,▇▂▃▅▅▄▃▁▂▃▄▃▆▂█▂▁▄▁▅▂▄▂▅▅▃▂▆▇▆▅▆▅▅▃▅▅▃▆▄
f1,0.46228
generation,The query selects th...
masked,False
nshot,0
precision,0.4059
query,select t1.celllabel ...


100%|██████████| 755/755 [00:00<00:00, 11605.20it/s]


f1,▅▃▅█▆▇▄▂▅▅▅▃▃▁▆▆▃▅▁▅█▅▄▇▄▄▁▅▇▇▇▆▇██▅█▇▆▃
masked,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nshot,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▅▄▇█▆▇▃▂▅▆▅▃▃▂▆▆▃▅▂▆█▅▄▇▅▅▁▅▆▇▆▆▇▇▇▄█▆▆▃
recall,▆▃▄▇▆▇▅▃▆▅▅▃▄▁▇▆▃▄▂▅▇▄▅▆▃▃▂▆█▇▇▆███▆█▇▆▃
f1,0.78732
generation,What were the top fi...
masked,False
nshot,1
precision,0.77277
query,select t1.celllabel ...


100%|██████████| 755/755 [00:00<00:00, 11600.74it/s]


...

f1,▅▅▁▇▆█▄▆▄▁▆▅▅▆▁▂▃▂▃▆▆▆▅▄▆▅▆▅▇▄▆▄▆▇▇▇▄▆▄▇
masked,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nshot,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▄▅▁▇▆█▃▅▄▁▆▅▆▆▂▃▂▂▃▅▆▆▄▄▇▅▇▅▇▅▆▅▅▇▇▇▅▆▄▇
recall,▆▆▂▇▆█▄▆▅▂▆▅▅▆▁▃▃▃▄▆▆▆▅▅▆▅▅▅▇▄▇▄▇▇▇█▄▆▄█
f1,0.8409
generation,what are the five mo...
masked,True
nshot,1
precision,0.77444
query,select t1.celllabel ...


100%|██████████| 755/755 [00:00<00:00, 11301.69it/s]


f1,▅▇▁█▂▅▆▃▁▃▇▄▄▄▆▂▅▂▆▆▆▅▆▇▇▇█▇▅▃▆▅▆▆▆▆▅▅▄▄
masked,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nshot,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▅▇▃█▄▆▆▄▁▃▇▆▅▄▇▃▅▄▆▆▇▆▇▇▇▇█▇▅▅▆▇▇▅▆▆▇▅▅▄
recall,▆▇▁▇▂▄▆▄▂▃▆▄▄▄▆▂▅▂▆▆▆▅▅▇▆▇█▇▆▃▇▄▇▆▆▆▃▅▄▅
f1,0.64794
generation,SELECT the top five ...
masked,True
nshot,2
precision,0.57391
query,select t1.celllabel ...


100%|██████████| 755/755 [00:00<00:00, 11251.54it/s]


f1,▆▇▁▇▆▅▆▆▅▃▇▃▅▄▄▁▄▂▃▇▇▄▅▇▆█▇▇▇▄▆▅▆▄▇▇▅▃▆▇
masked,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nshot,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▆█▁█▆▅▆▆▄▃█▃▄▄▅▁▅▄▂▇▇▅▄▇▇██▆▆▃▆▆▆▃▇▇▅▂▅▇
recall,▆▇▁▇▆▄▆▆▆▄▇▃▅▅▄▁▄▁▃▇▇▄▅▇▅█▇▇▇▄▇▅▇▅▇▇▅▄▆▆
f1,0.67451
generation,give me the top five...
masked,True
nshot,3
precision,0.60378
query,select t1.celllabel ...
